In [1]:
# Import the dependencies
import os
from dotenv import load_dotenv
import pymongo as pym
from pymongo import MongoClient
import pymongoarrow as pma
from pymongoarrow.api import write
import pandas as pd
import re

In [2]:
# load the config from the .env file
load_dotenv()
MONGODB_URI = os.environ['MONGODB_URI']

# Connect to the database engine
client = MongoClient(MONGODB_URI)

# connect to the project db
db = client['ExpectLifeRedux']

# get a reference to the data collection
data = db['ELR_Data']

In [3]:
# Create a dataframe from the collection
combined_df = pd.DataFrame(list(data.find()))
combined_df.head(5)

,_id,Country_Year,Country,Year,Gov Type,SSS Depth,SSS Type,Avg Rainfall (mm/yr),Pop Density (#/sq km),Total Population (M),...,Current HE per capita (PPP Intl $),Capital HE (% GDP),Domestic General Gov HE (% GDP),Domestic General Gov HE per capita (PPP Intl $),Domestic Private HE per capita (PPP Intl $),GDP growth per capita %,GDP per capita (PPP Intl $),Daily Caloric Supply,Tobacco use (% adults),Alcohol Use per capita (liters)
0,6588b0e7fc988d39759a6d37,Albania_1990,Albania,1990,parliamentary republic,2,"Universal medical benefits,Social insurance sy...",1485.0,119.946788,3.286542,...,222.786533,0.005188,2.765835,103.662764,115.435286,-11.187905,2549.746801,2568.0,35.0,6.57
1,6588b0e7fc988d39759a6d38,Albania_1991,Albania,1991,parliamentary republic,2,"Universal medical benefits,Social insurance sy...",1485.0,119.225912,3.266790,...,222.786533,0.005188,2.765835,103.662764,115.435286,-27.566821,1909.319160,2572.0,35.0,6.57
2,6588b0e7fc988d39759a6d39,Albania_1992,Albania,1992,parliamentary republic,2,"Universal medical benefits,Social insurance sy...",1485.0,118.505073,3.247039,...,222.786533,0.005188,2.765835,103.662764,115.435286,-6.622551,1823.503609,2654.0,35.0,6.57
3,6588b0e7fc988d39759a6d3a,Albania_1993,Albania,1993,parliamentary republic,2,"Universal medical benefits,Social insurance sy...",1485.0,117.784197,3.227287,...,222.786533,0.005188,2.765835,103.662764,115.435286,10.229949,2057.692048,2795.0,35.0,6.57
4,6588b0e7fc988d39759a6d3b,Albania_1994,Albania,1994,parliamentary republic,2,"Universal medical benefits,Social insurance sy...",1485.0,117.063358,3.207536,...,222.786533,0.005188,2.765835,103.662764,115.435286,8.969762,2290.143917,2877.0,35.0,6.57


In [4]:
# Drop the database id data and refresh the index
combined_df = combined_df.drop(['_id'], axis=1)
combined_df = combined_df.reset_index(drop=True)

# Set the DataFrame index to the country names to get them out of the way
combined_df= combined_df.set_index('Country_Year')
combined_df.head(5)

,Country,Year,Gov Type,SSS Depth,SSS Type,Avg Rainfall (mm/yr),Pop Density (#/sq km),Total Population (M),Total Labor Force (M),LEx years,...,Current HE per capita (PPP Intl $),Capital HE (% GDP),Domestic General Gov HE (% GDP),Domestic General Gov HE per capita (PPP Intl $),Domestic Private HE per capita (PPP Intl $),GDP growth per capita %,GDP per capita (PPP Intl $),Daily Caloric Supply,Tobacco use (% adults),Alcohol Use per capita (liters)
Country_Year,,,,,,,,,,,,,,,,,,,,,
Albania_1990,Albania,1990,parliamentary republic,2,"Universal medical benefits,Social insurance sy...",1485.0,119.946788,3.286542,1.374478,73.144,...,222.786533,0.005188,2.765835,103.662764,115.435286,-11.187905,2549.746801,2568.0,35.0,6.57
Albania_1991,Albania,1991,parliamentary republic,2,"Universal medical benefits,Social insurance sy...",1485.0,119.225912,3.266790,1.429833,73.378,...,222.786533,0.005188,2.765835,103.662764,115.435286,-27.566821,1909.319160,2572.0,35.0,6.57
Albania_1992,Albania,1992,parliamentary republic,2,"Universal medical benefits,Social insurance sy...",1485.0,118.505073,3.247039,1.438342,73.715,...,222.786533,0.005188,2.765835,103.662764,115.435286,-6.622551,1823.503609,2654.0,35.0,6.57
Albania_1993,Albania,1993,parliamentary republic,2,"Universal medical benefits,Social insurance sy...",1485.0,117.784197,3.227287,1.413557,73.939,...,222.786533,0.005188,2.765835,103.662764,115.435286,10.229949,2057.692048,2795.0,35.0,6.57
Albania_1994,Albania,1994,parliamentary republic,2,"Universal medical benefits,Social insurance sy...",1485.0,117.063358,3.207536,1.391914,74.131,...,222.786533,0.005188,2.765835,103.662764,115.435286,8.969762,2290.143917,2877.0,35.0,6.57


In [5]:
# Create a new DataFrame for the SSS data.
SSS_df = combined_df[['SSS Type']].copy()
SSS_df

,SSS Type
Country_Year,
Albania_1990,"Universal medical benefits,Social insurance sy..."
Albania_1991,"Universal medical benefits,Social insurance sy..."
Albania_1992,"Universal medical benefits,Social insurance sy..."
Albania_1993,"Universal medical benefits,Social insurance sy..."
Albania_1994,"Universal medical benefits,Social insurance sy..."
...,...
Zimbabwe_2018,Employer-liability system (cash sickness benef...
Zimbabwe_2019,Employer-liability system (cash sickness benef...
Zimbabwe_2020,Employer-liability system (cash sickness benef...


In [6]:
#create a list of SSS component phrases to match
sss_component_terms = ["community-based social insurance","employer-liability system (cash benefits)","employer-liability system (cash maternity benefits)","employer-liability system (cash medical benefits)","employer-liability system (cash sickness benefits)","employer-liability system (maternity benefits)","employer-liability system (medical benefits)","employer-liability system (parental leave)","employer-liability system (paternity benefits)","employer-liability system (paternity leave)","employer-liability system (sickness benefits)","employer-liability system through private carriers","employer-liability system","employment-related system (cash benefits)","employment-related system (cash maternity benefits)","employment-related system (cash sickness benefits)","employment-related system (parental benefits)","employment-related system (tuberculosis benefits)","mandatory health insurance (medical benefits)","mandatory health insurance through private carriers (medical benefits)","mandatory individual account system (medical benefits)","mandatory private insurance (cash benefits)","mandatory private insurance (medical benefits)","mandatory private insurance","mandatory private pension scheme","open pension funds","provident fund (birth grant)","provident fund (medical benefits)","Retirement pension scheme","social assistance system (birth grant)","social assistance system (cash benefits)","social assistance system (cash birth grants)","social assistance system (cash maternity benefits)","social assistance system (cash sickness benefits)","social assistance system (funeral grant)","social assistance system (long-term care benefits)","social assistance system (maternity benefits)","social assistance system (medical benefits)","social assistance system (parental leave)","social assistance system","social insurance system (cash benefits)","social insurance system (cash maternity benefits)","social insurance system (cash medical benefits)","social insurance system (cash parental benefits)","social insurance system (cash sickness benefits)","social insurance system (cash sickness)","social insurance system (child care benefits)","social insurance system (in-kind benefits)","social insurance system (maternity benefits)","social insurance system (maternity medical benefits)","social insurance system (medical benefits)","social insurance system (prenatal benefits)","social insurance system","universal (adoption grant)","universal (birth grant)","universal (cash maternity benefits)","universal (cash parental grants)","universal (paid parental leave)","universal (parental benefits)","universal (pregnancy grant)","universal (prenatal care grant)","universal medical benefits","voluntary community-based social insurance system (medical benefits)","voluntary pension scheme"]

In [7]:
# Create a regex pattern from the list of components that will match across word boundaries
regex_pattern = r'(' + '|'.join(map(re.escape, sorted(sss_component_terms, key=len, reverse=True))) + r')'

# Ensure the pattern is compiled for efficiency
compiled_pattern = re.compile(regex_pattern, flags=re.IGNORECASE)

# Function to match components using regex
def find_components(comp_type, pattern):
    # Find all matches of the pattern within the government_type string
    matches = re.findall(pattern, comp_type)
    return list(set(matches))  # return unique matches only

# Apply this function to the 'system_type' column
SSS_df['components_matched'] = SSS_df['SSS Type'].apply(lambda x: find_components(x, compiled_pattern))
SSS_df

,SSS Type,components_matched
Country_Year,,
Albania_1990,"Universal medical benefits,Social insurance sy...","[Social insurance system (cash benefits), Univ..."
Albania_1991,"Universal medical benefits,Social insurance sy...","[Social insurance system (cash benefits), Univ..."
Albania_1992,"Universal medical benefits,Social insurance sy...","[Social insurance system (cash benefits), Univ..."
Albania_1993,"Universal medical benefits,Social insurance sy...","[Social insurance system (cash benefits), Univ..."
Albania_1994,"Universal medical benefits,Social insurance sy...","[Social insurance system (cash benefits), Univ..."
...,...,...
Zimbabwe_2018,Employer-liability system (cash sickness benef...,[Employer-liability system (maternity benefits...
Zimbabwe_2019,Employer-liability system (cash sickness benef...,[Employer-liability system (maternity benefits...
Zimbabwe_2020,Employer-liability system (cash sickness benef...,[Employer-liability system (maternity benefits...


In [8]:
# components_matched contains lists of matched SSS component texts
sss_components_df = SSS_df['components_matched'].str.join('|').str.get_dummies('|')

# Concatenate with original df
sss_encoded_df = pd.concat([SSS_df, sss_components_df], axis=1)

In [9]:
# Drop the text columns
sss_encoded_df = sss_encoded_df.drop(['SSS Type','components_matched'], axis=1)
sss_encoded_df = sss_encoded_df.reset_index()
sss_encoded_df

,Country_Year,Community-based social insurance,Employer-liability system,Employer-liability system (cash benefits),Employer-liability system (cash maternity benefits),Employer-liability system (cash medical benefits),Employer-liability system (cash sickness benefits),Employer-liability system (maternity benefits),Employer-liability system (medical benefits),Employer-liability system (parental leave),...,Universal (birth grant),Universal (cash maternity benefits),Universal (cash parental grants),Universal (paid parental leave),Universal (parental benefits),Universal (pregnancy grant),Universal (prenatal care grant),Universal medical benefits,Voluntary community-based social insurance system (medical benefits),Voluntary pension scheme
0,Albania_1990,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,Albania_1991,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,Albania_1992,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Albania_1993,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Albania_1994,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6100,Zimbabwe_2018,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6101,Zimbabwe_2019,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6102,Zimbabwe_2020,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
6103,Zimbabwe_2021,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


### No further processing required - write the dataframe to the database ###

In [10]:
# load the config from the .env file
load_dotenv()
MONGODB_URI = os.environ['MONGODB_URI']

# Connect to the database engine
client = MongoClient(MONGODB_URI)

# connect to the project db
db = client['ExpectLifeRedux']

# get a reference to the data collection
SSS_data = db['Encoded_SSS_Data']

In [11]:
# prefered method - use PyMongoArrow
write(db.Encoded_SSS_Data, sss_encoded_df)

{'insertedCount': 6105}

In [12]:
# Create a new dataframe for the Government system data
gov_df = combined_df[['Gov Type']].copy()
gov_df

,Gov Type
Country_Year,
Albania_1990,parliamentary republic
Albania_1991,parliamentary republic
Albania_1992,parliamentary republic
Albania_1993,parliamentary republic
Albania_1994,parliamentary republic
...,...
Zimbabwe_2018,presidential republic
Zimbabwe_2019,presidential republic
Zimbabwe_2020,presidential republic


In [13]:
#create a list of component phrases to match
component_terms = ['absolute monarchy','communist state','constitutional federal republic','constitutional monarchy','dictatorship','federal parliamentary constitutional monarchy','federal parliamentary democracy under a constitutional monarchy','federal parliamentary republic','federal presidential republic','federal republic','in transition','mixed presidential-parliamentary system','parliamentary constitutional monarchy','parliamentary democracy under a constitutional monarchy','parliamentary democracy with limited self-government','parliamentary democracy','parliamentary republic','presidential limited democracy','presidential republic','previously parliamentary republic','semi-presidential federation','semi-presidential republic','theocratic republic','unitary parliamentary republic']

In [14]:
# Create a regex pattern from the list of components that will match across word boundaries
regex_pattern = r'(' + '|'.join(map(re.escape, sorted(component_terms, key=len, reverse=True))) + r')'

# Ensure the pattern is compiled for efficiency
compiled_pattern = re.compile(regex_pattern, flags=re.IGNORECASE)

# Function to match components using regex
def find_components(system_type, pattern):
    # Find all matches of the pattern within the system_type string
    matches = re.findall(pattern, system_type)
    return list(set(matches))  # return unique matches only

# Apply this function to the 'system_type' column
gov_df['components_matched'] = gov_df['Gov Type'].apply(lambda x: find_components(x, compiled_pattern))
gov_df

,Gov Type,components_matched
Country_Year,,
Albania_1990,parliamentary republic,[parliamentary republic]
Albania_1991,parliamentary republic,[parliamentary republic]
Albania_1992,parliamentary republic,[parliamentary republic]
Albania_1993,parliamentary republic,[parliamentary republic]
Albania_1994,parliamentary republic,[parliamentary republic]
...,...,...
Zimbabwe_2018,presidential republic,[presidential republic]
Zimbabwe_2019,presidential republic,[presidential republic]
Zimbabwe_2020,presidential republic,[presidential republic]


In [15]:
# components_matched contains lists of matched gov component texts
gov_components_df = gov_df['components_matched'].str.join('|').str.get_dummies('|')

# Concatenate with original df
gov_encoded_df = pd.concat([gov_df, gov_components_df], axis=1)

# Drop the text columns
gov_encoded_df = gov_encoded_df.drop(['Gov Type','components_matched'], axis=1)

gov_encoded_df = gov_encoded_df.reset_index()

In [16]:
gov_encoded_df

,Country_Year,absolute monarchy,communist state,constitutional federal republic,constitutional monarchy,dictatorship,federal parliamentary constitutional monarchy,federal parliamentary democracy under a constitutional monarchy,federal parliamentary republic,federal presidential republic,...,parliamentary democracy under a constitutional monarchy,parliamentary democracy with limited self-government,parliamentary republic,presidential limited democracy,presidential republic,previously parliamentary republic,semi-presidential federation,semi-presidential republic,theocratic republic,unitary parliamentary republic
0,Albania_1990,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,Albania_1991,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,Albania_1992,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,Albania_1993,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,Albania_1994,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6100,Zimbabwe_2018,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
6101,Zimbabwe_2019,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
6102,Zimbabwe_2020,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
6103,Zimbabwe_2021,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


### No further processing required - write the dataframe to the database ###

In [17]:
# load the config from the .env file
load_dotenv()
MONGODB_URI = os.environ['MONGODB_URI']

# Connect to the database engine
client = MongoClient(MONGODB_URI)

# connect to the project db
db = client['ExpectLifeRedux']

# get a reference to the data collection
gov_data = db['Encoded_Gov_Data']

In [18]:
# prefered method - use PyMongoArrow
write(db.Encoded_Gov_Data, gov_encoded_df)

{'insertedCount': 6105}